In [4]:
import spacy
from spacy.lang.tr import Turkish
from spacy.util import minibatch, compounding
import pandas as pd
import numpy as np

In [5]:
reviewspath = '/Users/pinarayaz/Jupyter/NLP/data/reviews_deasciified.csv'
reviews_df = pd.read_csv(reviewspath)
reviews_df.head()

,Review,Rating
0,nicholson gene harika,4.0
1,mükemmel derece kötü diyen arkadaşın sinema bi...,5.0
2,mükemmel derecede kötü bi film hep biselerin o...,1.0
3,nasıl beğendiğinizi anlamıyorum bu filmi filmd...,1.5
4,ok harika bir film senaryo gereğinden fazla ol...,5.0


In [6]:
nlp = Turkish()
sample_review = reviews_df.Review[2]
sample_review

'mükemmel derecede kötü bi film hep biselerin olmasını bekliyorsunuz bu filmden ama beklentileriniz boşa çıkıyor beğenenlere saygı duyarım ama ben hiç beğenmedim '

In [7]:
parsed_review = nlp(sample_review)
parsed_review

mükemmel derecede kötü bi film hep biselerin olmasını bekliyorsunuz bu filmden ama beklentileriniz boşa çıkıyor beğenenlere saygı duyarım ama ben hiç beğenmedim 

In [8]:
tokenized_text = pd.DataFrame()

for i, token in enumerate(parsed_review):
    tokenized_text.loc[i, 'text'] = token.text
    tokenized_text.loc[i, 'lemma'] = token.lemma_,
    tokenized_text.loc[i, 'pos'] = token.pos_
    tokenized_text.loc[i, 'tag'] = token.tag_
    tokenized_text.loc[i, 'dep'] = token.dep_
    tokenized_text.loc[i, 'shape'] = token.shape_
    tokenized_text.loc[i, 'is_alpha'] = token.is_alpha
    tokenized_text.loc[i, 'is_stop'] = token.is_stop
    tokenized_text.loc[i, 'is_punctuation'] = token.is_punct

tokenized_text

,text,lemma,pos,tag,dep,shape,is_alpha,is_stop,is_punctuation
0,mükemmel,mükemmel,,,,xxxx,True,False,False
1,derecede,"(derece,)",,,,xxxx,True,False,False
2,kötü,"(kötü,)",,,,xxxx,True,False,False
3,bi,"(bi,)",,,,xx,True,False,False
4,film,"(film,)",,,,xxxx,True,False,False
5,hep,"(hep,)",,,,xxx,True,True,False
6,biselerin,"(biselerin,)",,,,xxxx,True,False,False
7,olmasını,"(ol,)",,,,xxxx,True,False,False
8,bekliyorsunuz,"(bekle,)",,,,xxxx,True,False,False
9,bu,"(bu,)",,,,xx,True,True,False


In [9]:
reviews_df['tuples'] = reviews_df.apply(lambda row: (row['Review'],row['Rating']), axis=1)
train = reviews_df['tuples'].tolist()

In [10]:
nlp = spacy.blank("tr")  # create blank Language class

if "textcat" not in nlp.pipe_names:
    textcat = nlp.create_pipe("textcat", config={"exclusive_classes": True, "architecture": "simple_cnn"})
    nlp.add_pipe(textcat, last=True)
else:
    textcat = nlp.get_pipe("textcat")

In [11]:
textcat.add_label("POSITIVE")
textcat.add_label("NEGATIVE")

1

In [12]:
def load_data(limit=0, split=0.8):
    train_data = train
    np.random.shuffle(train_data)
    train_data = train_data[-limit:]
    texts, labels = zip(*train_data)
    cats = [{"POSITIVE": y >= 3.5, "NEGATIVE": y < 3.5} for y in labels]
    split = int(len(train_data) * split)
    return (texts[:split], cats[:split]), (texts[split:], cats[split:])

In [13]:
def evaluate(tokenizer, textcat, texts, cats):
    docs = (tokenizer(text) for text in texts)
    tp = 1e-8  # True positives
    fp = 1e-8  # False positives
    fn = 1e-8  # False negatives
    tn = 1e-8  # True negatives
    for i, doc in enumerate(textcat.pipe(docs)):
        gold = cats[i]
        for label, score in doc.cats.items():
            if label not in gold:
                continue
            if score >= 0.5 and gold[label] >= 0.5:
                tp += 1.
            elif score >= 0.5 and gold[label] < 0.5:
                fp += 1.
            elif score < 0.5 and gold[label] < 0.5:
                tn += 1
            elif score < 0.5 and gold[label] >= 0.5:
                fn += 1
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f_score = 2 * (precision * recall) / (precision + recall)
    accuracy = (tp + tn) / (tp + fp + fn + tn)
    return {'textcat_p': precision, 'textcat_r': recall, 'textcat_f': f_score, 'textcat_a': accuracy}

In [14]:
# load the dataset
lim = len(train)
print("Loading Reviews dataset...")
(train_texts, train_cats), (dev_texts, dev_cats) = load_data(limit=lim)
print("Using {} examples ({} training, {} evaluation)".format(lim, len(train_texts), len(dev_texts)))
train_data = list(zip(train_texts, [{'cats': cats} for cats in train_cats]))

Loading Reviews dataset...
Using 34990 examples (27992 training, 6998 evaluation)


In [15]:
# get names of other pipes to disable them during training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'textcat']
with nlp.disable_pipes(*other_pipes):  # only train textcat
    optimizer = nlp.begin_training()
    print("Training the model...")
    print('{:^5}\t{:^5}\t{:^5}\t{:^5}\t{:^5}'.format('LOSS', 'P', 'R', 'F', 'Accuracy'))
    for i in range(10):
        losses = {}
        # batch up the examples using spaCy's minibatch
        batches = minibatch(train_data, size=compounding(4., 32., 1.001))
        for batch in batches:
            texts, annotations = zip(*batch)
            nlp.update(texts, annotations, sgd=optimizer, drop=0.2, losses=losses)
        with textcat.model.use_params(optimizer.averages):
            # evaluate on the dev data split off in load_data()
            scores = evaluate(nlp.tokenizer, textcat, dev_texts, dev_cats)
        print('{0:.3f}\t{1:.3f}\t{2:.3f}\t{3:.3f}\t{4:.3f}'
              .format(losses['textcat'], scores['textcat_p'],
                      scores['textcat_r'], scores['textcat_f'], scores['textcat_a']))

Training the model...
LOSS 	  P  	  R  	  F  	Accuracy
15.616	0.773	0.773	0.773	0.773
12.076	0.774	0.774	0.774	0.774
9.166	0.775	0.775	0.775	0.775
6.944	0.770	0.770	0.770	0.770
5.544	0.770	0.770	0.770	0.770
4.268	0.767	0.767	0.767	0.767
3.837	0.763	0.763	0.763	0.763
3.248	0.762	0.762	0.762	0.762
2.707	0.758	0.758	0.758	0.758
2.455	0.759	0.759	0.759	0.759


In [16]:
# test the trained model
test_text = "çıktığı ilk gün izledim gerçekten efsane ama end game ' i izlediyseniz izleyin çünkü filmde end 'game ile ilgili ağır spolier veriyo tavsiye ederim baya güzeldi"
doc = nlp(test_text)
print(test_text, doc.cats)

çıktığı ilk gün izledim gerçekten efsane ama end game ' i izlediyseniz izleyin çünkü filmde end 'game ile ilgili ağır spolier veriyo tavsiye ederim baya güzeldi {'POSITIVE': 0.7114855647087097, 'NEGATIVE': 0.2885144352912903}


In [17]:
#save model
output_dir = '/Users/pinarayaz/Jupyter/NLP/spacy_models/reviews_spacy'
if output_dir is not None:
    with nlp.use_params(optimizer.averages):
        nlp.to_disk(output_dir)
    print("Saved model to", output_dir)

Saved model to /Users/pinarayaz/Jupyter/NLP/spacy_models/reviews_spacy


In [18]:
#test the saved model
test_text = "Kahramanın bütün güçlerini sergileyeceğiz derken senaryodan bu kadar ödün veren bu kadar fan servis film olamaz sadece 3 4 tane güzel sahnesi var sinemada izlemenizi tavsiye etmem."
nlp_test = spacy.load(output_dir)
doc2 = nlp_test(test_text)
print(test_text, doc2.cats)

score = 0
for cat in doc2.cats:
    if(doc2.cats[cat] > score):
        sentiment = cat
        score = doc2.cats[cat]
print("Sentiment:", sentiment)

Kahramanın bütün güçlerini sergileyeceğiz derken senaryodan bu kadar ödün veren bu kadar fan servis film olamaz sadece 3 4 tane güzel sahnesi var sinemada izlemenizi tavsiye etmem. {'POSITIVE': 0.028998078778386116, 'NEGATIVE': 0.971001923084259}
Sentiment: NEGATIVE
